In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))
sys.path.append(os.path.join(os.getcwd(), "molgen"))
sys.path.append(os.path.join(os.getcwd(), "iclr19-graph2graph/props"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN, fused_gw_torch
# from molgen.dataloading.MolGraphBatchPreprocessor import MolGraph
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, load_checkpoint, initialize_models
from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.ot_utils import encode_target
from mol_opt.ot_utils import FGW 
from mol_opt.ot_utils import compute_barycenter

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer
from molgen.metrics.Penalty import Penalty
from molgen.metrics.mol_metrics import MolMetrics

from rdkit.Chem import MolFromSmiles
from tensorboardX import SummaryWriter

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import grad
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from molgen.metrics.Penalty import Penalty as PenaltyNew

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

/home/octav/gitrepos/tum-thesis


In [3]:
model_type = "deepsets"
sys.argv = ["", "-model_type", model_type, "-one_batch_train"]
args = get_args()
outdir_suffix = "dev3_saves"
args.init_model = "{}-chembl1".format(model_type)
args.output_dir = "/run/media/octav/data-volume/tum-thesis/output_{}/{}/".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}/".format(outdir_suffix, args.init_model)

print (args.output_dir)
print (args.tb_logs_dir)

/run/media/octav/data-volume/tum-thesis/output_dev3_saves/deepsets-chembl1/
mol_opt/logs_dev3_saves/deepsets-chembl1/


In [6]:
model = args.init_model
model_iter = 11 

model_name = "model_{}_{}".format(model, model_iter)
print(args.output_dir + model_name)

# molopt, molopt_decoder, _, pen, recpen, _, _, config, _ = load_checkpoint(args.output_dir + model_name ,init_fc = initialize_models, device = 'cpu')
molopt, molopt_decoder, optimizer, penalty, recpenalty, crossatt, scheduler = initialize_models(args)

metrics = MolMetrics(SYMBOLS, FORMAL_CHARGES, BOND_TYPES, False, device = 'cpu')
fgw_loss = FGW(alpha = 0.5)
# pen = PenaltyNew(config, model_iter)

molopt_module_list = torch.nn.ModuleList([molopt, molopt_decoder])

# molopt = MolOpt(args)
# molopt_decoder = MolOptDecoder(args)
print (molopt)
print (molopt_decoder)
print(molopt.args.model_type)
pen.log()

/run/media/octav/data-volume/tum-thesis/output_dev3_saves/deepsets-chembl1/model_deepsets-chembl1_11
MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=200, bias=False)
    (W_message_h): Linear(in_features=200, out_features=200, bias=False)
    (W_message_o): Linear(in_features=293, out_features=150, bias=True)
    (W_mol_h): Linear(in_features=150, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt0): Linear(in_features=300, out_features=200, bias=True)
  (opt1): Linear(in_features=200, out_features=150, bias=True)
)
MolOptDecoder(
  (fc1_SYMBOLS): Linear(in_features=150, out_features=150, bias=True)
  (fc2_SYMBOLS): Linear(in_features=150, out_features=64, bias=True)
  (fc1_CHARGES): Linear(in_features=150, out_features=150, bias=True)
  (fc2_CHARGES): Linear(in_features=150, out_features=5, bias=True)
  (

In [7]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, True)
datatype = "val_split"
train_data_loader = get_loader("molgen/data/chembl50", datatype, 50, same_number_atoms = True)

for i in train_data_loader:
#     X = (MolGraph(i[0]))
#     Y = (MolGraph(i[1]))
    X = MolGraph(i)
    Y = X
    break

In [8]:
x_encoding, x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize_gumbel(*yhat_logits, tau = pen.tau)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

pen.mirror = ""

pen.tau = 0.05
fgw_loss_item = fgw_loss(*pred_pack, tau = 1)
print (fgw_loss_item.item()/args.batch_size)
pen_loss = pen(*pred_pack, model_iter)
print (pen_loss)
metrics.measure_batch(pred_pack[0], target)

/home/octav/gitrepos/tum-thesis/otgnn/models/gromov_modules.py:398: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  nce_reg = torch.nn.LogSoftmax()(torch.stack(all_nce_dists))[0]


3.6414093017578124
(tensor(0., grad_fn=<DivBackward0>), tensor(25998.8320, grad_fn=<DivBackward0>), tensor(9634.4160, grad_fn=<DivBackward0>))


({'batch_molecular_validity': 100.0,
  'batch_correctness': 0.0,
  'batch_symbol_accuracy': 0.0,
  'batch_molecular_disconnected_validity': 100.0,
  'batch_connected_components': 1.0,
  'batch_invalid_valency_nodes': 100.0,
  'batch_nodes_0degree': 1007,
  'batch_nodes_7plus_degree': 1007,
  'invalid_euler_toofew': 0.0,
  'invalid_euler_toomany': 98.0},
 {'avg_euler_error': (204.17999999999998, 50, 26766.267600000003),
  'batch_node_degree': (22.27606752730884, 1007, 60.30908960020668)})

In [9]:
nn.LogSoftmax(dim=1)(bonds_logits)

tensor([[-2.0000e+03, -2.0000e+03, -2.0000e+03, -2.0000e+03,  0.0000e+00],
        [-1.6877e+00, -1.6072e+00, -1.5906e+00, -1.5062e+00, -1.6657e+00],
        [-1.6934e+00, -1.6003e+00, -1.5968e+00, -1.5041e+00, -1.6632e+00],
        ...,
        [-1.6975e+00, -1.6066e+00, -1.5769e+00, -1.5156e+00, -1.6608e+00],
        [-1.6979e+00, -1.6006e+00, -1.5813e+00, -1.5149e+00, -1.6627e+00],
        [-2.0000e+03, -2.0000e+03, -2.0000e+03, -2.0000e+03,  0.0000e+00]],
       grad_fn=<LogSoftmaxBackward>)

In [10]:
bonds_logits

tensor([[-1.0000e+03, -1.0000e+03, -1.0000e+03, -1.0000e+03,  1.0000e+03],
        [-5.4358e-02,  2.6137e-02,  4.2725e-02,  1.2717e-01, -3.2361e-02],
        [-6.0568e-02,  3.2551e-02,  3.6042e-02,  1.2874e-01, -3.0361e-02],
        ...,
        [-6.3245e-02,  2.7605e-02,  5.7299e-02,  1.1864e-01, -2.6556e-02],
        [-6.5131e-02,  3.2255e-02,  5.1460e-02,  1.1791e-01, -2.9888e-02],
        [-1.0000e+03, -1.0000e+03, -1.0000e+03, -1.0000e+03,  1.0000e+03]],
       grad_fn=<CatBackward>)

In [11]:
M, pred_bonds_nll, target_bonds_rescaled = fgw_loss(*pred_pack, tau = 1)
num_atoms = M.shape[0]
alpha = 0.55
device = pred_bonds_nll.device

M=M
C1= target_bonds_rescaled
C2= -1 * pred_bonds_nll
p1=np.ones([num_atoms]) / float(num_atoms)
p2=np.ones([num_atoms]) / float(num_atoms)
dist_type='dot'
nce_reg = True
alpha=alpha
device=device

C1_detach = C1.detach().cpu().numpy()
C2_detach = C2.detach().cpu().numpy()
C1_m = max(1, C1_detach.max())
C2_m = max(1, C2_detach.max())
C1_detach /= C1_m
C2_detach /= C2_m
M_detach = M.detach().cpu().numpy()

from otgnn.models import np_fused_gw

ot_mat = np_fused_gw(M=M_detach, C1 = C1_detach, C2 = C2_detach,
                     p1 = p1, p2 = p2, dist_type=dist_type, alpha=alpha)
print (ot_mat.shape)

TypeError: iteration over a 0-d tensor

In [27]:
yhat_embedding = molopt.GCN(Y)[0]
print (x_embedding.shape, yhat_embedding.shape)

torch.Size([1007, 150]) torch.Size([1007, 150])


In [29]:
k0 = nn.Linear(args.pc_hidden, args.n_hidden).to(device = args.device)
k1 = nn.Linear(args.n_hidden, common_dim).to(device = args.device)
q0 = nn.Linear(args.pc_hidden, args.n_hidden).to(device = args.device)
q1 = nn.Linear(args.n_hidden, common_dim).to(device = args.device)

In [37]:
common_dim = 200
eps = 1e-7
# k = torch.autograd.Variable(torch.randn(args.pc_hidden, common_dim))
# q = torch.autograd.Variable(torch.randn(args.pc_hidden, common_dim))

for idx, (stx, lex) in enumerate(X.scope):
    yhat = yhat_embedding[stx:stx+lex]
    x = x_embedding[stx:stx+lex]
#     M = 1/np.sqrt(common_dim) * torch.matmul(torch.matmul(x, k), torch.matmul(q.T, yhat.T))
    M = torch.matmul(k1(F.leaky_relu(k0(x))), q1(F.leaky_relu(q0(yhat))).T)
    attn = torch.softmax(M, dim = 1) + eps
    W = (attn / attn.sum(axis = 0))
   
    print (idx, lex)
    print (W.shape)
    print (W.sum(axis = 0))
    print (W.sum(axis = 1))
    break

0 17
torch.Size([17, 17])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


In [42]:
for i in k0.parameters():
    print (i)

Parameter containing:
tensor([[-0.0700, -0.0383, -0.0717,  ...,  0.0210,  0.0813,  0.0446],
        [ 0.0548, -0.0314,  0.0514,  ...,  0.0368, -0.0411,  0.0807],
        [ 0.0478,  0.0359, -0.0662,  ...,  0.0024,  0.0054, -0.0631],
        ...,
        [ 0.0699, -0.0390,  0.0456,  ..., -0.0803,  0.0784,  0.0790],
        [-0.0016,  0.0233,  0.0094,  ..., -0.0279,  0.0154, -0.0731],
        [-0.0447,  0.0185,  0.0551,  ..., -0.0683, -0.0563, -0.0248]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0167,  0.0268, -0.0133,  0.0802, -0.0004,  0.0639,  0.0809,  0.0652,
        -0.0780, -0.0300,  0.0614, -0.0001, -0.0441, -0.0648,  0.0264, -0.0286,
         0.0109,  0.0613,  0.0081, -0.0486,  0.0341,  0.0414, -0.0525,  0.0342,
         0.0790,  0.0410,  0.0710,  0.0742, -0.0435,  0.0531,  0.0010,  0.0106,
        -0.0558,  0.0281,  0.0700, -0.0667, -0.0313,  0.0618, -0.0235,  0.0710,
        -0.0076, -0.0444,  0.0473, -0.0512, -0.0128,  0.0778,  0.0607, -0.0812,
        -0.0750

In [47]:
q1(F.leaky_relu(q0(yhat))).shape

torch.Size([17, 200])

In [48]:
k1(F.leaky_relu(k0(x))).shape

torch.Size([17, 200])

In [49]:
torch.matmul(k1(F.leaky_relu(k0(x))), q1(F.leaky_relu(q0(yhat))).T)

tensor([[0.0286, 0.0779, 0.0858, 0.0302, 0.0597, 0.0630, 0.0734, 0.0594, 0.0589,
         0.0836, 0.0289, 0.0583, 0.0761, 0.0421, 0.0275, 0.0281, 0.0399],
        [0.0283, 0.0747, 0.0830, 0.0308, 0.0594, 0.0627, 0.0692, 0.0561, 0.0556,
         0.0801, 0.0267, 0.0564, 0.0729, 0.0405, 0.0257, 0.0262, 0.0383],
        [0.0244, 0.0711, 0.0807, 0.0272, 0.0565, 0.0597, 0.0665, 0.0532, 0.0520,
         0.0755, 0.0247, 0.0528, 0.0694, 0.0365, 0.0218, 0.0224, 0.0342],
        [0.0306, 0.0796, 0.0877, 0.0320, 0.0624, 0.0660, 0.0755, 0.0614, 0.0604,
         0.0852, 0.0306, 0.0602, 0.0782, 0.0443, 0.0294, 0.0299, 0.0420],
        [0.0263, 0.0745, 0.0829, 0.0289, 0.0576, 0.0607, 0.0690, 0.0556, 0.0551,
         0.0790, 0.0256, 0.0544, 0.0719, 0.0387, 0.0247, 0.0252, 0.0367],
        [0.0267, 0.0747, 0.0831, 0.0291, 0.0577, 0.0609, 0.0692, 0.0559, 0.0553,
         0.0792, 0.0258, 0.0546, 0.0721, 0.0389, 0.0247, 0.0253, 0.0368],
        [0.0271, 0.0724, 0.0816, 0.0296, 0.0576, 0.0608, 0.0668, 0.053

In [35]:
M.clamp(-1, 1)

tensor([[0.0286, 0.0779, 0.0858, 0.0302, 0.0597, 0.0630, 0.0734, 0.0594, 0.0589,
         0.0836, 0.0289, 0.0583, 0.0761, 0.0421, 0.0275, 0.0281, 0.0399],
        [0.0283, 0.0747, 0.0830, 0.0308, 0.0594, 0.0627, 0.0692, 0.0561, 0.0556,
         0.0801, 0.0267, 0.0564, 0.0729, 0.0405, 0.0257, 0.0262, 0.0383],
        [0.0244, 0.0711, 0.0807, 0.0272, 0.0565, 0.0597, 0.0665, 0.0532, 0.0520,
         0.0755, 0.0247, 0.0528, 0.0694, 0.0365, 0.0218, 0.0224, 0.0342],
        [0.0306, 0.0796, 0.0877, 0.0320, 0.0624, 0.0660, 0.0755, 0.0614, 0.0604,
         0.0852, 0.0306, 0.0602, 0.0782, 0.0443, 0.0294, 0.0299, 0.0420],
        [0.0263, 0.0745, 0.0829, 0.0289, 0.0576, 0.0607, 0.0690, 0.0556, 0.0551,
         0.0790, 0.0256, 0.0544, 0.0719, 0.0387, 0.0247, 0.0252, 0.0367],
        [0.0267, 0.0747, 0.0831, 0.0291, 0.0577, 0.0609, 0.0692, 0.0559, 0.0553,
         0.0792, 0.0258, 0.0546, 0.0721, 0.0389, 0.0247, 0.0253, 0.0368],
        [0.0271, 0.0724, 0.0816, 0.0296, 0.0576, 0.0608, 0.0668, 0.053

In [35]:
M.shape

torch.Size([25, 25])

In [37]:
M

tensor([[-4.8806e+01,  1.2745e+02,  8.5013e+01,  1.6420e+02,  1.8269e+01,
         -1.2526e+02, -6.7990e+01,  2.9985e+01, -6.0364e+01, -6.3572e+01,
          2.8890e+01, -6.3572e+01, -6.0364e+01,  2.9985e+01, -5.5506e+01,
         -1.0067e+02,  5.0763e+01,  2.1775e+02,  9.6130e+01,  1.8065e+02,
         -3.5103e+01,  2.1741e+02,  1.9373e+02,  1.2703e+02,  5.9381e+01],
        [-1.8349e+01,  9.2621e+01,  1.3784e+02,  1.6916e+02,  8.9967e+00,
         -1.7360e+02, -1.5138e+02, -4.8960e+01, -1.5916e+02, -1.6577e+02,
         -4.5353e+01, -1.6577e+02, -1.5916e+02, -4.8960e+01, -1.4277e+02,
         -1.3320e+02,  5.1016e+01,  9.9278e+01,  8.4654e+01,  1.2031e+02,
         -9.5534e+01,  1.5498e+02,  1.2172e+02,  5.0620e+01,  4.9736e+01],
        [ 1.3419e+01,  1.7614e+02,  2.3679e+02,  3.0860e+02,  2.7714e+01,
         -1.3887e+02, -7.2370e+01, -6.2119e+01, -8.8330e+01, -9.7444e+01,
         -6.0173e+01, -9.7444e+01, -8.8330e+01, -6.2119e+01, -6.4899e+01,
         -8.9064e+01,  7.9825e+01,  

In [48]:
Mdet_orig = M.detach().cpu().numpy().copy()
Mdet = M.detach().cpu().numpy().copy()
np.random.shuffle(Mdet)

In [50]:
Mdet

array([[-1.36550903e+02,  6.81647415e+01,  1.03288399e+02,
         7.67897720e+01,  3.60319481e+01, -1.35173340e+02,
        -1.08069221e+02,  3.83839455e+01, -8.44185638e+01,
        -8.56652298e+01,  4.37059669e+01, -8.56652298e+01,
        -8.44185638e+01,  3.83839607e+01, -8.79000778e+01,
        -9.98548050e+01,  4.25657501e+01,  2.22053623e+01,
         5.34014587e+01,  1.11797241e+02,  8.24223709e+01,
         2.01263702e+02,  6.49279480e+01,  7.15621567e+01,
         8.77687454e+01],
       [-1.41745499e+02,  6.95025024e+01,  1.02559906e+02,
         7.86224289e+01,  3.32511063e+01, -1.30562500e+02,
        -1.09084831e+02,  3.38087959e+01, -8.56264267e+01,
        -8.72580566e+01,  3.84175034e+01, -8.72580566e+01,
        -8.56264267e+01,  3.38088036e+01, -8.87138443e+01,
        -9.84396439e+01,  4.13976135e+01,  2.88886242e+01,
         5.95451431e+01,  1.15779877e+02,  8.77940063e+01,
         2.05081360e+02,  6.82530594e+01,  7.96451645e+01,
         8.66283646e+01],
    

In [52]:
Mdet_orig

array([[-1.36550903e+02,  6.81647415e+01,  1.03288399e+02,
         7.67897720e+01,  3.60319481e+01, -1.35173340e+02,
        -1.08069221e+02,  3.83839455e+01, -8.44185638e+01,
        -8.56652298e+01,  4.37059669e+01, -8.56652298e+01,
        -8.44185638e+01,  3.83839607e+01, -8.79000778e+01,
        -9.98548050e+01,  4.25657501e+01,  2.22053623e+01,
         5.34014587e+01,  1.11797241e+02,  8.24223709e+01,
         2.01263702e+02,  6.49279480e+01,  7.15621567e+01,
         8.77687454e+01],
       [-5.45599861e+01,  1.00158745e+02,  1.05964836e+02,
        -2.31788368e+01,  1.03137474e+02,  1.13762985e+02,
         4.22908745e+01,  1.18958099e+02,  5.50460625e+01,
         5.07430649e+01,  1.22109154e+02,  5.07430649e+01,
         5.50460625e+01,  1.18958084e+02,  3.96638756e+01,
         1.58115173e+02,  1.54708847e+02, -8.90717983e+00,
         1.79497879e+02,  1.27680420e+02,  2.79696488e+00,
         7.15046005e+01,  1.03312851e+02,  7.64186859e+01,
         4.51117783e+01],
    

In [53]:
M[torch.randperm(M.size()[0])].detach().clone()

tensor([[-1.0118e+02, -1.5469e+01, -5.3189e+01, -4.8951e+01, -1.4664e+02,
         -3.8039e+02, -2.6164e+02, -8.3566e+01, -2.3841e+02, -2.3828e+02,
         -9.4402e+01, -2.3828e+02, -2.3841e+02, -8.3566e+01, -2.4270e+02,
         -3.4588e+02, -8.8649e+01,  7.1412e+01, -6.5000e+01, -1.1127e+02,
          1.5097e+01,  2.0600e+02,  1.3323e+01,  1.3850e+01,  1.6531e+02],
        [-5.4560e+01,  1.0016e+02,  1.0596e+02, -2.3179e+01,  1.0314e+02,
          1.1376e+02,  4.2291e+01,  1.1896e+02,  5.5046e+01,  5.0743e+01,
          1.2211e+02,  5.0743e+01,  5.5046e+01,  1.1896e+02,  3.9664e+01,
          1.5812e+02,  1.5471e+02, -8.9072e+00,  1.7950e+02,  1.2768e+02,
          2.7970e+00,  7.1505e+01,  1.0331e+02,  7.6419e+01,  4.5112e+01],
        [-1.4175e+02,  6.9503e+01,  1.0256e+02,  7.8622e+01,  3.3251e+01,
         -1.3056e+02, -1.0908e+02,  3.3809e+01, -8.5626e+01, -8.7258e+01,
          3.8418e+01, -8.7258e+01, -8.5626e+01,  3.3809e+01, -8.8714e+01,
         -9.8440e+01,  4.1398e+01,  

In [15]:
attn.T / attn.sum(axis = 0)

tensor([[1.8529e-02, 2.5068e-07, 3.9987e-02, 4.0000e-02, 4.0000e-02, 4.0000e-02,
         4.0000e-02, 3.9179e-02, 4.0000e-02, 4.0000e-02, 1.8434e-07, 4.0000e-02,
         4.0000e-02, 3.9179e-02, 4.0000e-02, 1.0009e-07, 4.0000e-02, 9.9163e-08,
         4.0000e-02, 9.5670e-09, 2.4148e-08, 4.0000e-02, 1.0542e-06, 1.7043e-06,
         2.1951e-08],
        [1.8529e-02, 2.5068e-07, 3.9987e-02, 4.0000e-02, 4.0000e-02, 4.0000e-02,
         4.0000e-02, 3.9179e-02, 4.0000e-02, 4.0000e-02, 1.8434e-07, 4.0000e-02,
         4.0000e-02, 3.9179e-02, 4.0000e-02, 1.0009e-07, 4.0000e-02, 9.9163e-08,
         4.0000e-02, 9.5670e-09, 2.4148e-08, 1.5957e+05, 3.5173e-08, 1.7043e-06,
         2.1951e-08],
        [1.8529e-02, 2.5068e-07, 4.0323e-02, 4.0000e-02, 4.0000e-02, 4.0000e-02,
         4.0000e-02, 3.9179e-02, 4.0000e-02, 4.0000e-02, 1.8434e-07, 4.0000e-02,
         4.0000e-02, 3.9179e-02, 4.0000e-02, 1.0009e-07, 4.0000e-02, 9.9163e-08,
         4.0000e-02, 9.5670e-09, 2.4148e-08, 4.0000e-02, 3.5173e-

In [16]:
(attn / attn.sum(axis = 0)

SyntaxError: unexpected EOF while parsing (<ipython-input-16-bfc19aa26183>, line 1)

In [17]:
attn.sum(axis = 0)


tensor([5.3970e-06, 3.9892e-01, 2.5008e-06, 2.5000e-06, 2.5000e-06, 2.5000e-06,
        2.5000e-06, 2.5524e-06, 2.5000e-06, 2.5000e-06, 5.4248e-01, 2.5000e-06,
        2.5000e-06, 2.5524e-06, 2.5000e-06, 9.9907e-01, 2.5000e-06, 1.0084e+00,
        2.5000e-06, 1.0453e+01, 4.1411e+00, 2.5000e-06, 2.8431e+00, 5.8674e-02,
        4.5557e+00], grad_fn=<SumBackward1>)

In [20]:
torch.nn.ModuleList([None])

ModuleList(
  (0): None
)